### Contributor: Sudesh Kumar Santhosh Kumar
- Email: sudeshkumar.santhoshkumar@lexisnexis.com
- Date: 21st June, 2023

Description:  This Notebook will show you how to fine-tune BERT for multi-class text classification tasks, meaning that there are more than just two classes in the dataset. We'll be using the 20 Newsgroups dataset, which is a test classification dataset with 20 different classes. 